<a href="https://colab.research.google.com/github/tahahah/model-pruning-experiments/blob/master/notebooks/pruning_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Pruning Experiments

This notebook demonstrates pruning experiments on:
1. Vision Transformer (ViT)
2. Deep Compression AutoEncoder (DC-AE)

In [1]:
# Clone the repository
!git clone https://github.com/tahahah/model-pruning-experiments.git
%cd model-pruning-experiments

# Install dependencies
!pip install -r requirements.txt

Cloning into 'model-pruning-experiments'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 22 (delta 2), reused 22 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (22/22), 429.69 KiB | 5.44 MiB/s, done.
Resolving deltas: 100% (2/2), done.
/content/model-pruning-experiments
  Cloning https://github.com/mit-han-lab/efficientvit.git to /tmp/pip-req-build-qwmsjsf3
  Running command git clone --filter=blob:none --quiet https://github.com/mit-han-lab/efficientvit.git /tmp/pip-req-build-qwmsjsf3
  Resolved https://github.com/mit-han-lab/efficientvit.git to commit b94ff779828eea399c78f626b574da2d50ef2e49
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/alibaba/TinyNeuralNetwork.git to /tmp/pip-install-7erzmjmi/tinyneuralnetwork_7855b343fa4643f48dbacc991c588c08
  Running c

## 1. Vision Transformer (ViT) Pruning

In [ ]:
from vit_pruning_analysis import *

# Run ViT pruning analysis
main()

## 2. DC-AE Pruning

In [6]:
!git pull
!python vae_pruning_analysis.py

CUDA available: True
CUDA device: Tesla T4
Loading DC-AE model...


TypeError: DCAE_HF.__init__() got an unexpected keyword argument 'use_cuda'

## View Results

The results are saved in the `output` directory:
- Pruning analysis reports
- Reconstructed images
- Model statistics